In [1]:
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import nltk
# Скачать русские стоп-слова
# Téléchargez les stopwords russes
nltk.download("stopwords")

# Загрузите и прочитайте текстовый файл
# Charger et lire le fichier texte
file_path = "news.txt"
with open(file_path, "r", encoding="utf-8") as f:
    texts = f.readlines()

# Инициализируем морфологический анализатор и стоп-слова
# Initialiser l'analyseur morphologique et les stopwords
morph = MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

# Функция предварительной обработки
# Fonction de prétraitement
def preprocess_text(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)  # Tokenisation de base
    tokens = [morph.normal_forms(word)[0] for word in tokens if word not in stop_words]  # Lemmatisation et suppression des stopwords
    return tokens

# Предварительная обработка корпуса
# Prétraitement du corpus
processed_corpus = [preprocess_text(text) for text in texts]

# Обучение модели Word2Vec
# Entraîner le modèle Word2Vec
model = Word2Vec(sentences=processed_corpus, vector_size=100, window=5, min_count=2, workers=4)

# Сохранить модель
# Sauvegarder le modèle
model_path = "word2vec_russian.model"
model.save(model_path)

print(f"Modèle entraîné et sauvegardé sous {model_path}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moumi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modèle entraîné et sauvegardé sous word2vec_russian.model


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec

model_path = "word2vec_russian.model"
model = Word2Vec.load(model_path)

# Функция для вычисления среднего значения векторов слов для документа
# Fonction pour calculer la moyenne des vecteurs de mots pour un document
def document_to_vector(document, model):
    vectors = []
    for word in document:
        if word in model.wv:  # Vérifier si le mot est dans le vocabulaire
            vectors.append(model.wv[word])
    if vectors:  # Si des vecteurs sont trouvés
        return np.mean(vectors, axis=0)
    else:  # Si aucun vecteur n'est trouvé
        return np.zeros(model.vector_size)

# Разделить тексты на обучающий и тестовый наборы
# Diviser les textes en ensembles d'entraînement et de test
train_texts, test_texts = train_test_split(processed_corpus, test_size=0.2, random_state=42)

# Преобразование текстов в векторы
# Mapper les textes en vecteurs
train_vectors = np.array([document_to_vector(doc, model) for doc in train_texts])
test_vectors = np.array([document_to_vector(doc, model) for doc in test_texts])

# Проверка
# Vérification
print("Taille de l'ensemble d'entraînement :", train_vectors.shape)
print("Taille de l'ensemble de test :", test_vectors.shape)


Taille de l'ensemble d'entraînement : (8000, 100)
Taille de l'ensemble de test : (2000, 100)


In [3]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Пример надписей к текстам
# текста связаны с категориями/классом (например, 0, 1 и т. д.)
# Пример: train_labels = [0, 1, 1, 0, ...] и test_labels = [0, 1, ...]
# Exemple d'étiquettes pour les textes 
# le textes sont associés à des catégories/classe (par ex., 0, 1, etc.)
# Exemple : train_labels = [0, 1, 1, 0, ...] et test_labels = [0, 1, ...]
train_labels = np.random.randint(0, 2, size=len(train_vectors))  # Remplacez par vos vraies étiquettes
test_labels = np.random.randint(0, 2, size=len(test_vectors))    # Remplacez par vos vraies étiquettes

# Обучение модели SVM
# Entraîner un modèle SVM
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(train_vectors, train_labels)

# Прогнозирование классов на тестовом наборе
# Prédire les classes sur l'ensemble de test
predicted_labels = svm_model.predict(test_vectors)

# Оцените производительность
# Évaluer les performances
print("Classification Report:")
print(classification_report(test_labels, predicted_labels))

print("Accuracy Score:", accuracy_score(test_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.46      0.49      1016
           1       0.50      0.56      0.53       984

    accuracy                           0.51      2000
   macro avg       0.51      0.51      0.51      2000
weighted avg       0.51      0.51      0.51      2000

Accuracy Score: 0.508


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

# Создаем представление TF-IDF
# Construire une représentation TF-IDF
texts_joined = [" ".join(doc) for doc in processed_corpus]  # Joindre les tokens pour chaque document
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(texts_joined)

# Извлекаем слова и их веса TF-IDF
# Récupérer les mots et leurs poids TF-IDF
vocab = tfidf.get_feature_names_out()
word2tfidf = dict(zip(vocab, tfidf.idf_))

# Функция для расчета взвешенных векторов по TF-IDF
# Fonction pour calculer les vecteurs pondérés par TF-IDF
def document_to_tfidf_vector(document, model, word2tfidf):
    vectors = []
    for word in document:
        if word in model.wv and word in word2tfidf:  # Vérifier si le mot est dans le vocabulaire et dans TF-IDF
            vectors.append(model.wv[word] * word2tfidf[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Рассчитать взвешенные векторы для всех документов
# Calculer les vecteurs pondérés pour l'ensemble des documents
train_vectors_tfidf = np.array([document_to_tfidf_vector(doc, model, word2tfidf) for doc in train_texts])
test_vectors_tfidf = np.array([document_to_tfidf_vector(doc, model, word2tfidf) for doc in test_texts])

# Нормализовать векторы
# Normaliser les vecteurs
train_vectors_tfidf = normalize(train_vectors_tfidf)
test_vectors_tfidf = normalize(test_vectors_tfidf)


In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Создание помеченных документов
# Créer des documents étiquetés
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(processed_corpus)]

# Обучение модели Doc2Vec
# Entraîner le modèle Doc2Vec
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=2, workers=4, epochs=20)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Получить векторы документов
# Obtenir les vecteurs des documents
train_vectors_doc2vec = np.array([doc2vec_model.dv[str(i)] for i in range(len(train_texts))])
test_vectors_doc2vec = np.array([doc2vec_model.dv[str(i)] for i in range(len(test_texts))])


In [6]:
# Реформируйте алгоритм SVM с помощью взвешенных векторов TF-IDF или Doc2Vec.
# Reformez l'algorithme SVM avec les vecteurs TF-IDF pondérés ou Doc2Vec
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(train_vectors_tfidf, train_labels)  # ou train_vectors_doc2vec

# Прогнозирование классов на тестовом наборе
# Prédire les classes sur l'ensemble de test
predicted_labels = svm_model.predict(test_vectors_tfidf)  # ou test_vectors_doc2vec

# Оцените производительность
# Évaluer les performances
print("Classification Report:")
print(classification_report(test_labels, predicted_labels))

print("Accuracy Score:", accuracy_score(test_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.28      0.36      1016
           1       0.49      0.72      0.58       984

    accuracy                           0.49      2000
   macro avg       0.50      0.50      0.47      2000
weighted avg       0.50      0.49      0.47      2000

Accuracy Score: 0.4945
